In [ ]:
# Execute when running on Google Colab
# !pip install neograd

In [48]:
# Execute when running locally
import sys
sys.path.append('../../tests')
import _setup

In [49]:
import neograd as ng
import numpy as np
from neograd.nn.loss import BCE
from neograd.nn.optim import Adam
from neograd.nn.utils import get_batches
from sklearn.datasets import load_digits

In [50]:
X_train,_ = load_digits(return_X_y=True) # load data

In [51]:
num_train = X_train.shape[0] # number of train examples
num_iter = 200 # number of iterations
batch_size = 200 # batch size in each iteration

In [52]:
# normalize inputs
X_train_norm = (X_train - np.mean(X_train, axis=1, keepdims=True))/np.std(X_train, axis=1, keepdims=True)

In [53]:
X_train = ng.tensor(X_train_norm)

In [54]:
class NN(ng.nn.Model):
    def __init__(self):
        self.generator = ng.nn.Sequential(
            ng.nn.Linear(64,100),
            ng.nn.ReLU(),
            ng.nn.Linear(100,80),
            ng.nn.ReLU(),
            ng.nn.Linear(80,64)
        )
        self.discriminator = ng.nn.Sequential(
            ng.nn.Linear(64,50),
            ng.nn.ReLU(),
            ng.nn.Linear(50,25),
            ng.nn.ReLU(),
            ng.nn.Linear(25,1),
            ng.nn.Sigmoid()
        )
    
    def forward(self, inputs):
        noise = ng.tensor(np.random.randn(num_train, 64))
        generated = self.generator(noise)
        discriminator_fake = self.discriminator(generated)
        discriminator_real = self.discriminator(inputs)
        return discriminator_fake, discriminator_real

In [55]:
model = NN()

model.discriminator.freeze()
optim_generator = Adam(model.get_params(), 5e-3)
model.discriminator.unfreeze()

model.generator.freeze()
optim_discriminator = Adam(model.get_params(), 5e-3)
model.generator.freeze()

loss_fn = BCE()

In [56]:
def train(discriminator_fake, discriminator_real):
    optim_generator.zero_grad()
    optim_discriminator.zero_grad()
    loss_generator = loss_fn(discriminator_fake, ng.tensor(np.ones((num_train,1))))
    loss_generator.backward(retain_graph=True) # retain_graph is True to retain all components that lead to discriminator_fake
    optim_generator.step()

    optim_generator.zero_grad()
    optim_discriminator.zero_grad()
    loss_real = loss_fn(discriminator_real, ng.tensor(np.ones((num_train,1))))
    loss_fake = loss_fn(discriminator_fake, ng.tensor(np.zeros((num_train,1))))
    loss_real.backward(retain_graph=True)
    loss_fake.backward()
    optim_discriminator.step()

In [57]:
discriminator_fake, discriminator_real = model(X_train)

In [58]:
train(discriminator_fake, discriminator_real)